In [32]:
import pandas as pd
import os
from datetime import datetime

In [10]:
trnx_folder = "./ledgered/ledgered_app/resources/raw_transactions/"
os.listdir(trnx_folder)

['raw_fidelity_large.csv',
 'Chase8140_2023.CSV',
 'Chase_sample.CSV',
 'raw_fidelity.csv',
 'raw_fidelity_test.csv']

In [35]:
OUTPUT_SCHEMA = {
            "date": "object",
            "type": "string",
            "amount": "float64",
            "original_description": "string"
        }

In [22]:
def snake_case(s):
    """Convert strings to snake case"""
    return s.replace(" ", "_").lower()

In [23]:
def sign_to_type(number):
    if float(number) > 0:
        return "Credit"
    else:
        return "Debit"

In [24]:
chase_df = pd.read_csv(trnx_folder + 'Chase8140_2023.CSV')

In [25]:
chase_df.columns = [snake_case(colname) for colname in chase_df.columns]

In [36]:
def process_raw_transaction_df(df):
        """Expected Output:
        {
            "date": "object",
            "type": "string",
            "amount": "float64",
            "original_description": "string"
        }
        """
        processed_df = df.copy()
        processed_df["original_description"] = processed_df["description"].astype("string")
        # -/+ amount is debit/credit
        processed_df["type"] = processed_df["amount"].apply(lambda x: sign_to_type(x)).astype("string")
        processed_df["amount"] = processed_df["amount"].astype(float).apply(abs)
        processed_df["date"] = processed_df["transaction_date"].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date())
        return processed_df[OUTPUT_SCHEMA.keys()]

In [38]:
chase_processed = process_raw_transaction_df(chase_df)

In [ ]:
# how to consolidate!!!!

In [43]:
len(chase_processed)

172

In [46]:
chase_processed.groupby(['date','type','original_description'], as_index=False).amount.sum()

,date,type,original_description,amount
0,2022-12-29,Debit,AMERICAN AIR0010280316665,30.00
1,2022-12-29,Debit,DCA SAYSIBON PIER C,3.70
2,2022-12-29,Debit,MEZEH - DCA,16.94
3,2022-12-30,Debit,BIG NIGHT LIVE,23.40
4,2022-12-30,Debit,FAMILY PRACTICE GROUP PC,20.00
...,...,...,...,...
146,2023-02-13,Debit,OPENART AI,10.00
147,2023-02-13,Debit,SQ *NINE BAR ESPRESSO,4.50
148,2023-02-14,Debit,OPENART AI,2.00
149,2023-02-15,Credit,Payment Thank You - Web,2412.25
